<a href="https://colab.research.google.com/github/denisangelo/Federated_Learning/blob/main/FL_Not_Division_Dataset_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#Importanto Pytorch, Flower,Opacus
!pip install -q flwr[simulation]
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install opacus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
#Importanto as bibliotecas necessárias
from collections import OrderedDict
from typing import Dict, Optional, List, Tuple
import matplotlib.pyplot as plt

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from flwr.common import Metrics
import torchvision.transforms as transforms
import opacus
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from torch.optim.adam import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

In [14]:
#Definindo o dispositivo de treinamento
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE} using Pytorch{torch.__version__} and Flower{fl.__version__}")

Training on cpu using Pytorch2.0.1+cu118 and Flower1.4.0


In [15]:
# Definindo os valores das variáveis
BATCH_SIZE = 1024
NUM_CLIENTS= 10
optim_lr=0.002
local_epochs=5
num_rounds=10

In [16]:
# Carregando os dados
def load_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = CIFAR10(root="./data", train=True,download=True, transform=transform)
    testset = CIFAR10(root="./data", train=False, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
    testloader = DataLoader(testset,batch_size=BATCH_SIZE,shuffle=False,num_workers=0)
    num_examples = {"trainset": len(trainset)*BATCH_SIZE, "testset": len(testset)*BATCH_SIZE}

    return trainloader, testloader, num_examples

In [17]:
# Definindo o Modelo
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [18]:
# Definindo as funções de treino e teste
def train(net, trainloader, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    for _ in range(epochs):
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()

def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [19]:
# Funções de atualização de parâmetros do modelo no servidor agregador
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [20]:

# Definindo a classe de cliente Virtual
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, testloader,num_examples) -> None:
        super().__init__()
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader
        self.num_examples = num_examples


    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader,epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}


    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.testloader)
        return float(loss), len(self.testloader), {"accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
        net = Net().to(DEVICE)
        trainloader, testloader, num_examples = load_data()
  
        return FlowerClient(cid, net, trainloader, testloader,num_examples)

In [21]:
# Realizando a agregação
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [22]:
# Iniciando o treino federado
# Criando a Função de agregação FedAvg
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,#-->Fração de clientes necessários para treinamento
    fraction_evaluate=1.0, #--> Fração de clientes que serão utilizados para teste
    min_fit_clients=NUM_CLIENTS, #--> Número mínimo que serão utlilizados para treinamento
    min_evaluate_clients=2, #--> Número mínimo de clientes que serão realizados para teste
    min_available_clients=NUM_CLIENTS,#--> Número mínimo de clientes que serão utilizados no treinamento
    evaluate_metrics_aggregation_fn=weighted_average,  
)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Inicia a simulação
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-10 02:15:31,238 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


(launch_and_evaluate pid=16184) Files already downloaded and verified


2023-06-10 02:15:36,892	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-10 02:15:40,756 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 7811353806.0, 'object_store_memory': 3905676902.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7811353806.0, 'object_store_memory': 3905676902.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO flwr 2023-06-10 02:15:40,764 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-10 02:15:40,766 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=19533) 2023-06-10 02:15:49.559304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_get_parameters pid=19533) Files already downloaded and verified
(launch_and_get_parameters pid=19533) Files already downloaded and verified


INFO flwr 2023-06-10 02:15:56,713 | server.py:277 | Received initial parameters from one random client
INFO:flwr:Received initial parameters from one random client
INFO flwr 2023-06-10 02:15:56,720 | server.py:88 | Evaluating initial parameters
INFO:flwr:Evaluating initial parameters
INFO flwr 2023-06-10 02:15:56,727 | server.py:101 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2023-06-10 02:15:56,730 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)
DEBUG:flwr:fit_round 1: strategy sampled 10 clients (out of 10)


...

(launch_and_fit pid=19534) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=19534) Files already downloaded and verified
(launch_and_fit pid=19533) [Client 1] fit, config: {}
(launch_and_fit pid=19533) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_fit pid=19534) Files already downloaded and verified
(launch_and_fit pid=19533) [Client 0] fit, config: {}
(launch_and_fit pid=19533) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_fit pid=19534) Files already downloaded and verified
(launch_and_fit pid=19533) [Client 8] fit, config: {}
(launch_and_fit pid=19533) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_fit pid=19534) Files already downloaded and verified
(launch_and_fit pid=19533) [Client 4] fit, config: {}
(launch_and_fit pid=19534) Files already downloaded and verified [repeated 2x across cluster]
(launch_and_fit pid=19534) [Client 2] fit, config

DEBUG flwr 2023-06-10 06:12:13,694 | server.py:232 | fit_round 10 received 10 results and 0 failures
DEBUG:flwr:fit_round 10 received 10 results and 0 failures
DEBUG flwr 2023-06-10 06:12:13,734 | server.py:168 | evaluate_round 10: strategy sampled 10 clients (out of 10)
DEBUG:flwr:evaluate_round 10: strategy sampled 10 clients (out of 10)


(launch_and_evaluate pid=19533) Files already downloaded and verified [repeated 2x across cluster]
(launch_and_evaluate pid=19533) [Client 8] evaluate, config: {}
(launch_and_evaluate pid=19533) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=19534) Files already downloaded and verified
(launch_and_evaluate pid=19533) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=19533) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=19534) Files already downloaded and verified
(launch_and_evaluate pid=19533) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=19533) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=19534) Files already downloaded and verified
(launch_and_evaluate pid=19533) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=19533) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=19534) Files a

DEBUG flwr 2023-06-10 06:13:12,404 | server.py:182 | evaluate_round 10 received 10 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 10 results and 0 failures
INFO flwr 2023-06-10 06:13:12,408 | server.py:147 | FL finished in 14235.678013433
INFO:flwr:FL finished in 14235.678013433
INFO flwr 2023-06-10 06:13:12,410 | app.py:218 | app_fit: losses_distributed [(1, 0.0014769753813743593), (2, 0.00132684246301651), (3, 0.0012242138266563413), (4, 0.0011602699875831607), (5, 0.001117008435726166), (6, 0.0010843732237815857), (7, 0.001061166560649872), (8, 0.0010418044686317446), (9, 0.0010251726329326633), (10, 0.00101297749876976)]
INFO:flwr:app_fit: losses_distributed [(1, 0.0014769753813743593), (2, 0.00132684246301651), (3, 0.0012242138266563413), (4, 0.0011602699875831607), (5, 0.001117008435726166), (6, 0.0010843732237815857), (7, 0.001061166560649872), (8, 0.0010418044686317446), (9, 0.0010251726329326633), (10, 0.00101297749876976)]
INFO flwr 2023-06-10 06:13:12,412 | app

History (loss, distributed):
	round 1: 0.0014769753813743593
	round 2: 0.00132684246301651
	round 3: 0.0012242138266563413
	round 4: 0.0011602699875831607
	round 5: 0.001117008435726166
	round 6: 0.0010843732237815857
	round 7: 0.001061166560649872
	round 8: 0.0010418044686317446
	round 9: 0.0010251726329326633
	round 10: 0.00101297749876976
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.46599999999999997), (2, 0.521), (3, 0.5613), (4, 0.5877000000000001), (5, 0.6087), (6, 0.6155999999999999), (7, 0.6236000000000002), (8, 0.6317), (9, 0.6396000000000001), (10, 0.6452)]}